# W2 Practicals - COMP4702
### (Supervised Learning)

### Aims:
* To complement lecture material in understanding unsupervised learning.
* To produce some assessable work for this subject.

In [ ]:
# Common Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# ANSI color
class color:
  YELLOW_BOLD = '\033[1;33m'
  END = '\033[0m'

### Q1 Apply the k-means clustering algorithm (as implemented in Matlab or python scikit-learn) to the heightweight dataset (second and third columns). Plot the resulting cluster centres together with the data. Colour the data according to class label (first column; hint: sort the data!).